# Exercise: Error correcting Neural Networks

This is a rather long exercise where you will put most of what you have learned during the exercises to good use. The majority of it can be done without use of the LEGO robot so you can finish on your own time if you do not finish today. 

During this exercise we will train an error correcting neural network. The method is similar to that employed by [Anatole v. Lilienfeld's group for electronic structure calculations of molecules](https://pubs.acs.org/doi/10.1021/acs.jctc.5b00099) ([arxiv](https://arxiv.org/abs/1503.04987)). Generating data is rather expensive with the LEGO robots, and we cannot mix data from all 4 robots without inducing significant noise. We thus have to make as much as possible out of the precious data we have.

You will:
 * Change the SilicoColorMixer model to be as similar to the LEGO as possible
 * Train a neural network to predict the error between the updated SilicoColorMixer and AiLEGO_master (the robot) color mixer
 * Test performance 

Import all the modules and functions you think you will be needing today.

In [ ]:
# import statements here

We start by making sure you can actually do the last thing. Right now, most of the data you have on the robots are correlated as they originate from oprimization runs. We want to use some of this as a test dataset (dependent test set) but we also want a test dataset that is not correlated with the optimization runs. Run the robot to generate 9 data points where you decide the input list, like you did on the first robot day. This will be your independent data set. Use the cells below. You have done this before and can do it again; this time unassisted.

In [ ]:
# for you to generate independent test data set on robot

In [ ]:
# for you to generate independent test data set on robot

In [ ]:
# for you to generate independent test data set on robot. Add more cells as needed

## Change SilicoColorMixer

Change the color mixer such that it is as similar to the LEGO robot as possible without changing the source code. By make sure the `SilicoColorMixer` and the robot are as similar as possible, you can use a smaller neural network to predict the error and require less training data.

First, lets change the rgb codes of the used colors in the `SilicoColorMixer`. If you cannot remember how to do this, go back and reread the documentation of the `SilicoColorMixer` or open the DocString by initializing and pressing `Shift`+`Tab` while having the cursor inside the cell below.

In [ ]:
from silico_color_mixer import SilicoColorMixer

Initialize a mixer. We have plenty of noise in our lego data already so you can remove that from the silico mixer. Change the colors such that the color codes are that of the pure colors you have obtained with your robot. You should have those in your robot data. Pay attention to the input formats. You can find examples in the DocString.

In [ ]:
# Cell for you to change colors used in SilicoColorMixer and initialize the mixer.

In [ ]:
# Cell for you to change colors used in SilicoColorMixer and initialize the mixer. Add more as needed.

Try out your need updated silico mixer to make sure it works.

In [ ]:
# Cell for you to try out the mixer. Add more cells as needed.

## Optimization of weights

During the robot runs you might have noticed that the different colors have different strength. A little blue is very potent while you need a lot of yellow for the outcome to get a yellow tone. This difference in strength is also reflected in the measured rgb color codes. Try to update your silico model to reflect this by adding weights to each color input. 

It can be done using a wrapper function as the one below.

In [ ]:
def run_cuvette_w_weights(colors_input):
    w_r, w_g, w_b, w_y = 1., 1., 1., 1.
    weighted_colors = [colors_input[0] * w_r,
                       colors_input[1] * w_g,
                       colors_input[2] * w_b,
                       colors_input[3] * w_y]
    return mixer.run_cuvette(weighted_colors)

In [ ]:
run_cuvette_w_weights([1.,1.,1.,1.])

In the above example, all the weights, `w_r` ect., are `1.`. You need to optimize those. Go find your robot data and find a datapoint with input `[0.25, 0.25, 0.25, 0.25]`. You should all have at least one of those. Reinitialize your silico mixer giving the rgb code you obtained from that datapoint as target. Based on this single data point and what you have learned so far, optimize the weights. You will need to write a new wrapper function that takes the weights as input.

If you look closer at the `[0.25, 0.25, 0.25, 0.25]` data point, you will likely find that the values in the rgb code are so low that they cannot possibly be achived by a linear combination of the pure colors. Life is not linear, and when you physically mix all four colors they counteract each other. The resulting color is a blueish semitransparent grey with a low rgb signal. Your fit will thus not be all that great.

In [ ]:
# Cell for you to work in.

In [ ]:
# Cell for you to work in.

In [ ]:
# Cell for you to work in.

In [ ]:
# Cell for you to work in. Add more as needed

Do the weights you find during optimization make sense? How sensitive are these weights to changes in the input colors and the target color? 

The noise in our robot data is too large to use it to improve the silico model by incorporating weights into it. We would need to use more data points and proably also a more advanced model than simply adding weights. Lets see if we can close the gaps with an error correcting neural network rather than attempting to improve the silico model further. I.e., do not use the mixer with weights moving forward but do change the input colors to those you obtained with the robot.

## Error correcting neural network

The other day you trained a neural network to replicate the silico mixer. Today is slightly different as you will need it to predict the error between the robot and the silico mixer. Start by getting your data from the robot and splitting it into train set, dependent test set, and independent test set.

Start by copying the robot data file to your own data folder. It cannot easily be done from within the notebook so make a new login to gbar. From gbar run the below line. Insert your robot IP.

Use pandas to load in the data. It might perhaps be a bit overkill, but pandas is a great data science tool for you to get an introduction to if you don't already know it.

In [ ]:
try:
    import pandas as pd
except:
    !pip3 install pandas --user --upgrade
    import pandas as pd

You can then load in the data with the cell below

In [ ]:
df = pd.read_csv('~/47332/data/datafile.csv', sep=';')
print(df)

In [ ]:
data = df[['Vcolors','RGB']]
print(data)

Split the independent data you created from the dataframe. In case it is the last 9 points use the one below. Otherwise you have to change the indicies.

In [ ]:
df1 = data.iloc[:-9, :]
df_indep_test = data.iloc[-9:, :]

# In case n points after the independent set
# n = <your number of points>
# df11 = date.iloc[:-9-n, :]
# df_independent = date.iloc[-9-n:-n, :]
# df12 = date.iloc[-n:, :]
# df1 = pd.concat([df11, df12])

Remove (on average) 20 percent of the points in the remaining data. These points will be the dependent test data.

In [ ]:
msk = np.random.rand(len(df1)) < 0.8
df_train = df1[msk]
df_dep_test = df1[~msk]
print(len(df_train))
print(len(df_dep_test))

Now, turn the dataframes into lists. You can use `df_train.values.tolist()` and similar.

In [ ]:
train_list = df_train.values.tolist()
for data in train_list:
    data[0] = eval(data[0])
    data[1] = list(eval(data[1]))
# print(train_list)
train_x = list(np.asarray(train_list)[:,0])
# print(train_x)
train_y = np.asarray(train_list)[:,1]
# print(train_y)

In [ ]:
# For you to work in

In [ ]:
# For you to work in

You will not be training to the robot data but to the difference between the robot and the silico mixer. Go ahead and make the difference list. Remember that in the end it should have the format `[[r0,g0,b0],[r1,g1,b2],...]`.

You might find numpy usefull as it allows elementwise difference between numpy arrays to be easily calculated. 
`list(np.asarray(point)-np.asarray(point_sil))`

In [ ]:
# For you to work in

Train a neural network like you did the other day.

In [ ]:
# For you to work in

You can now predict what the LEGO robot will yield by the sum of the silico mixer prediction and the error correcting neural network. Try to see how well it does on both the dependent and independent test sets.

In [ ]:
# For you to work in

In [ ]:
# For you to work in

In [ ]:
# For you to work in

In [ ]:
# For you to work in. Add cells as needed

Are you satisfied with the results? Do you have an idea as to how you could improve the predictive power? How many data points do you need in the training to get good results? You can change the number of training points indirectly by changing the value in the `msk` used to split data into the train set and dependent test set.